# **Kaggle – DataTops®**
Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes
- Última submission:
    - Mañana: 17 de febrero a las 5pm
    - Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica:
El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## 1. Librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import urllib.request

## 2. Datos

In [2]:
# ============================================================================
# PASO 1: CARGAR DATOS (CORRECTAMENTE)
# ============================================================================
print("="*70)
print("PASO 1: CARGA DE DATOS")
print("="*70)

import pandas as pd
import numpy as np

# Cargar archivos
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

print(f"\n✅ Datos cargados:")
print(f"   train: {train.shape} (CON precio)")
print(f"   test: {test.shape} (SIN precio)")
print(f"   sample_submission: {sample_submission.shape}")

# Copias para procesar
train_processed = train.copy()
test_processed = test.copy()


PASO 1: CARGA DE DATOS

✅ Datos cargados:
   train: (912, 13) (CON precio)
   test: (391, 12) (SIN precio)
   sample_submission: (391, 2)


### 2.1 Exploración de los datos

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_in_euros    912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [4]:
print(train.columns.tolist())

['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_in_euros']


In [5]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [6]:
train.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
907,28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
908,1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
909,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
910,23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
911,229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [7]:
train.describe()

,laptop_ID,Inches,Price_in_euros
count,912.000000,912.000000,912.000000
mean,650.312500,14.981579,1111.724090
std,382.727748,1.436719,687.959172
min,2.000000,10.100000,174.000000
25%,324.750000,14.000000,589.000000
50%,636.500000,15.600000,978.000000
75%,982.250000,15.600000,1483.942500
max,1320.000000,18.400000,6099.000000


In [ ]:
# Valores nulos
print("\n VALORES NULOS EN TRAIN:")
nulos_train = train.isnull().sum()
if nulos_train.sum() > 0:
    print(nulos_train[nulos_train > 0].sort_values(ascending=False))
else:
    print("    No hay valores nulos")

print("\n VALORES NULOS EN TEST:")
nulos_test = test.isnull().sum()
if nulos_test.sum() > 0:
    print(nulos_test[nulos_test > 0].sort_values(ascending=False))
else:
    print("    No hay valores nulos")

# Tipos de columnas
print("\n TIPOS DE COLUMNAS:")
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

# Excluir id y target
if 'id' in numeric_cols:
    numeric_cols.remove('id')
if 'Price_euros' in numeric_cols:
    numeric_cols.remove('Price_euros')

print(f"\n   Numéricas ({len(numeric_cols)}):")
for col in numeric_cols:
    print(f"      - {col}")

print(f"\n   Categóricas ({len(categorical_cols)}):")
for col in categorical_cols:
    n_unique = train[col].nunique()
    print(f"      - {col:30s} ({n_unique} valores únicos)")


🔍 VALORES NULOS EN TRAIN:
   ✅ No hay valores nulos

🔍 VALORES NULOS EN TEST:
   ✅ No hay valores nulos

📊 TIPOS DE COLUMNAS:

   Numéricas (3):
      - laptop_ID
      - Inches
      - Price_in_euros

   Categóricas (10):
      - Company                        (19 valores únicos)
      - Product                        (480 valores únicos)
      - TypeName                       (6 valores únicos)
      - ScreenResolution               (36 valores únicos)
      - Cpu                            (107 valores únicos)
      - Ram                            (9 valores únicos)
      - Memory                         (37 valores únicos)
      - Gpu                            (93 valores únicos)
      - OpSys                          (9 valores únicos)
      - Weight                         (165 valores únicos)


In [ ]:
print("="*70)
print("PASO 2: FEATURE ENGINEERING")
print("="*70)

# Hacer copias para no modificar originales
train_processed = train.copy()
test_processed = test.copy()

print("\n🔧 PROCESAMIENTO DE FEATURES...")

# ============================================================================
# 1. RAM: Convertir "8GB" → 8
# ============================================================================
print("\n1️⃣ Procesando RAM...")

def extract_ram(ram_str):
    """Extrae número de GB de RAM. Ejemplo: '8GB' → 8"""
    if pd.isna(ram_str):
        return np.nan
    # Extraer números
    import re
    numbers = re.findall(r'\d+', str(ram_str))
    if numbers:
        return int(numbers[0])
    return np.nan

train_processed['Ram_GB'] = train_processed['Ram'].apply(extract_ram)
test_processed['Ram_GB'] = test_processed['Ram'].apply(extract_ram)

print(f"    Ram original: {train['Ram'].head(3).tolist()}")
print(f"    Ram_GB: {train_processed['Ram_GB'].head(3).tolist()}")

# ============================================================================
# 2. WEIGHT: Convertir "1.86kg" → 1.86
# ============================================================================
print("\n2️ Procesando Weight...")

def extract_weight(weight_str):
    """Extrae peso en kg. Ejemplo: '1.86kg' → 1.86"""
    if pd.isna(weight_str):
        return np.nan
    import re
    # Buscar números (incluyendo decimales)
    numbers = re.findall(r'\d+\.?\d*', str(weight_str))
    if numbers:
        return float(numbers[0])
    return np.nan

train_processed['Weight_kg'] = train_processed['Weight'].apply(extract_weight)
test_processed['Weight_kg'] = test_processed['Weight'].apply(extract_weight)

print(f"    Weight original: {train['Weight'].head(3).tolist()}")
print(f"    Weight_kg: {train_processed['Weight_kg'].head(3).tolist()}")

# ============================================================================
# 3. MEMORY: Extraer tipo y capacidad
# ============================================================================
print("\n3️ Procesando Memory...")

def extract_memory_features(memory_str):
    """
    Extrae características de memoria.
    Ejemplos:
    - '256GB SSD' → (256, 1, 0)  # (capacidad, es_ssd, es_hdd)
    - '1TB HDD' → (1024, 0, 1)
    - '128GB SSD +  1TB HDD' → (1152, 1, 1)  # Híbrido
    """
    if pd.isna(memory_str):
        return (np.nan, 0, 0)
    
    memory_str = str(memory_str).upper()
    
    # Detectar SSD y HDD
    has_ssd = 1 if 'SSD' in memory_str else 0
    has_hdd = 1 if 'HDD' in memory_str else 0
    
    # Extraer capacidades
    import re
    total_gb = 0
    
    # Buscar patrones como "256GB", "1TB"
    gb_matches = re.findall(r'(\d+)\s*GB', memory_str)
    tb_matches = re.findall(r'(\d+)\s*TB', memory_str)
    
    for gb in gb_matches:
        total_gb += int(gb)
    
    for tb in tb_matches:
        total_gb += int(tb) * 1024
    
    return (total_gb if total_gb > 0 else np.nan, has_ssd, has_hdd)

# Aplicar
memory_features = train_processed['Memory'].apply(extract_memory_features)
train_processed['Memory_GB'] = [x[0] for x in memory_features]
train_processed['Has_SSD'] = [x[1] for x in memory_features]
train_processed['Has_HDD'] = [x[2] for x in memory_features]

memory_features_test = test_processed['Memory'].apply(extract_memory_features)
test_processed['Memory_GB'] = [x[0] for x in memory_features_test]
test_processed['Has_SSD'] = [x[1] for x in memory_features_test]
test_processed['Has_HDD'] = [x[2] for x in memory_features_test]

print(f"    Memory original: {train['Memory'].head(3).tolist()}")
print(f"    Memory_GB: {train_processed['Memory_GB'].head(3).tolist()}")
print(f"    Has_SSD: {train_processed['Has_SSD'].head(3).tolist()}")
print(f"    Has_HDD: {train_processed['Has_HDD'].head(3).tolist()}")

# ============================================================================
# 4. SCREEN RESOLUTION: Extraer resolución y tipo
# ============================================================================
print("\n4️⃣ Procesando ScreenResolution...")

def extract_screen_features(screen_str):
    """
    Extrae características de pantalla.
    Ejemplos:
    - 'Full HD 1920x1080' → (1920, 1080, 1, 0)  # (width, height, is_fhd, is_4k)
    - '4K Ultra HD 3840x2160' → (3840, 2160, 0, 1)
    """
    if pd.isna(screen_str):
        return (np.nan, np.nan, 0, 0, 0)
    
    screen_str = str(screen_str).upper()
    
    # Detectar tipos
    is_touchscreen = 1 if 'TOUCHSCREEN' in screen_str else 0
    is_ips = 1 if 'IPS' in screen_str else 0
    is_4k = 1 if '4K' in screen_str or '3840' in screen_str else 0
    
    # Extraer resolución (patrón: 1920x1080)
    import re
    resolution = re.search(r'(\d+)\s*x\s*(\d+)', screen_str)
    
    if resolution:
        width = int(resolution.group(1))
        height = int(resolution.group(2))
        total_pixels = width * height
    else:
        width, height, total_pixels = np.nan, np.nan, np.nan
    
    return (total_pixels, is_touchscreen, is_ips, is_4k)

screen_features = train_processed['ScreenResolution'].apply(extract_screen_features)
train_processed['Screen_Pixels'] = [x[0] for x in screen_features]
train_processed['Is_Touchscreen'] = [x[1] for x in screen_features]
train_processed['Is_IPS'] = [x[2] for x in screen_features]
train_processed['Is_4K'] = [x[3] for x in screen_features]

screen_features_test = test_processed['ScreenResolution'].apply(extract_screen_features)
test_processed['Screen_Pixels'] = [x[0] for x in screen_features_test]
test_processed['Is_Touchscreen'] = [x[1] for x in screen_features_test]
test_processed['Is_IPS'] = [x[2] for x in screen_features_test]
test_processed['Is_4K'] = [x[3] for x in screen_features_test]

print(f"    ScreenResolution original: {train['ScreenResolution'].head(2).tolist()}")
print(f"    Screen_Pixels: {train_processed['Screen_Pixels'].head(2).tolist()}")
print(f"    Is_Touchscreen: {train_processed['Is_Touchscreen'].head(2).tolist()}")

# ============================================================================
# 5. CPU: Extraer marca y tipo
# ============================================================================
print("\n5️⃣ Procesando CPU...")

def extract_cpu_brand(cpu_str):
    """Extrae marca de CPU"""
    if pd.isna(cpu_str):
        return 'Unknown'
    cpu_str = str(cpu_str).upper()
    if 'INTEL CORE I7' in cpu_str:
        return 'Intel_i7'
    elif 'INTEL CORE I5' in cpu_str:
        return 'Intel_i5'
    elif 'INTEL CORE I3' in cpu_str:
        return 'Intel_i3'
    elif 'AMD' in cpu_str:
        return 'AMD'
    elif 'INTEL' in cpu_str:
        return 'Intel_Other'
    else:
        return 'Other'

train_processed['CPU_Brand'] = train_processed['Cpu'].apply(extract_cpu_brand)
test_processed['CPU_Brand'] = test_processed['Cpu'].apply(extract_cpu_brand)

print(f"    CPU original: {train['Cpu'].head(2).tolist()}")
print(f"    CPU_Brand: {train_processed['CPU_Brand'].head(2).tolist()}")

# ============================================================================
# 6. GPU: Extraer marca
# ============================================================================
print("\n6️⃣ Procesando GPU...")

def extract_gpu_brand(gpu_str):
    """Extrae marca de GPU"""
    if pd.isna(gpu_str):
        return 'Unknown'
    gpu_str = str(gpu_str).upper()
    if 'NVIDIA' in gpu_str or 'GEFORCE' in gpu_str:
        return 'Nvidia'
    elif 'AMD' in gpu_str or 'RADEON' in gpu_str:
        return 'AMD'
    elif 'INTEL' in gpu_str:
        return 'Intel'
    else:
        return 'Other'

train_processed['GPU_Brand'] = train_processed['Gpu'].apply(extract_gpu_brand)
test_processed['GPU_Brand'] = test_processed['Gpu'].apply(extract_gpu_brand)

print(f"    GPU original: {train['Gpu'].head(2).tolist()}")
print(f"    GPU_Brand: {train_processed['GPU_Brand'].head(2).tolist()}")

print("\n FEATURE ENGINEERING COMPLETADO")

PASO 2: FEATURE ENGINEERING

🔧 PROCESAMIENTO DE FEATURES...

1️⃣ Procesando RAM...
   ✅ Ram original: ['8GB', '16GB', '8GB']
   ✅ Ram_GB: [8, 16, 8]

2️⃣ Procesando Weight...
   ✅ Weight original: ['1.86kg', '2.59kg', '2.04kg']
   ✅ Weight_kg: [1.86, 2.59, 2.04]

3️⃣ Procesando Memory...
   ✅ Memory original: ['256GB SSD', '1TB HDD', '1TB HDD']
   ✅ Memory_GB: [256.0, 1024.0, 1024.0]
   ✅ Has_SSD: [1, 0, 0]
   ✅ Has_HDD: [0, 1, 1]

4️⃣ Procesando ScreenResolution...
   ✅ ScreenResolution original: ['Full HD 1920x1080', 'Full HD 1920x1080']
   ✅ Screen_Pixels: [nan, nan]
   ✅ Is_Touchscreen: [0, 0]

5️⃣ Procesando CPU...
   ✅ CPU original: ['Intel Core i3 6006U 2GHz', 'Intel Core i7 6700HQ 2.6GHz']
   ✅ CPU_Brand: ['Intel_i3', 'Intel_i7']

6️⃣ Procesando GPU...
   ✅ GPU original: ['Intel HD Graphics 520', 'Nvidia GeForce GTX 960<U+039C>']
   ✅ GPU_Brand: ['Intel', 'Nvidia']

✅ FEATURE ENGINEERING COMPLETADO


In [10]:
print("="*70)
print("FEATURE ENGINEERING AVANZADO")
print("="*70)

import numpy as np

# Aplicar a train y test
for df in [train_processed, test_processed]:
    
    # 1. PPI (Pixels Per Inch)
    df['PPI'] = df['Screen_Pixels'] / (df['Inches'] ** 2)
    df['PPI'] = df['PPI'].fillna(df['PPI'].median())
    
    # 2. RAM por peso
    df['RAM_per_Weight'] = df['Ram_GB'] / df['Weight_kg']
    df['RAM_per_Weight'] = df['RAM_per_Weight'].fillna(df['RAM_per_Weight'].median())
    
    # 3. Storage por peso
    df['Storage_per_Weight'] = df['Memory_GB'] / df['Weight_kg']
    df['Storage_per_Weight'] = df['Storage_per_Weight'].fillna(df['Storage_per_Weight'].median())
    
    # 4. Total de recursos
    df['Total_Resources'] = df['Ram_GB'] + (df['Memory_GB'] / 100)
    
    # 5. Es gaming
    df['Is_Gaming'] = ((df['GPU_Brand'] == 'Nvidia') & (df['Ram_GB'] >= 8)).astype(int)
    
    # 6. Es premium
    df['Is_Premium'] = (
        (df['Company'] == 'Apple') | 
        ((df['Ram_GB'] >= 16) & (df['Has_SSD'] == 1))
    ).astype(int)
    
    # 7. Interacción RAM x Storage
    df['RAM_x_Storage'] = df['Ram_GB'] * np.log1p(df['Memory_GB'])
    df['RAM_x_Storage'] = df['RAM_x_Storage'].fillna(df['RAM_x_Storage'].median())

print("✅ Nuevas features creadas")

FEATURE ENGINEERING AVANZADO
✅ Nuevas features creadas


C:\Users\lupep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\lupep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [11]:
print("\n" + "="*70)
print("VERIFICACIÓN DE NUEVAS FEATURES")
print("="*70)

# Ver nuevas columnas numéricas
nuevas_numericas = ['Ram_GB', 'Weight_kg', 'Memory_GB', 'Has_SSD', 'Has_HDD', 
                    'Screen_Pixels', 'Is_Touchscreen', 'Is_IPS', 'Is_4K']

print("\n📊 NUEVAS FEATURES NUMÉRICAS:")
print(train_processed[nuevas_numericas].describe())

# Ver nuevas categóricas
nuevas_categoricas = ['CPU_Brand', 'GPU_Brand']

print("\n📊 NUEVAS FEATURES CATEGÓRICAS:")
for col in nuevas_categoricas:
    print(f"\n{col}:")
    print(train_processed[col].value_counts())


VERIFICACIÓN DE NUEVAS FEATURES

📊 NUEVAS FEATURES NUMÉRICAS:
           Ram_GB   Weight_kg    Memory_GB     Has_SSD     Has_HDD  \
count  912.000000  912.000000   906.000000  912.000000  912.000000   
mean     8.263158    2.026937   602.388521    0.643640    0.434211   
std      5.044788    0.665466   471.401399    0.479186    0.495925   
min      2.000000    0.690000     8.000000    0.000000    0.000000   
25%      4.000000    1.490000   256.000000    0.000000    0.000000   
50%      8.000000    2.040000   500.000000    1.000000    0.000000   
75%      8.000000    2.300000  1024.000000    1.000000    1.000000   
max     64.000000    4.700000  2560.000000    1.000000    1.000000   

       Screen_Pixels  Is_Touchscreen      Is_IPS       Is_4K  
count            0.0      912.000000  912.000000  912.000000  
mean             NaN        0.143640    0.275219    0.028509  
std              NaN        0.350917    0.446870    0.166513  
min              NaN        0.000000    0.000000    0.

In [ ]:
print("="*70)
print("CORRECCIÓN: SCREEN_PIXELS")
print("="*70)

# El problema es el regex. Vamos a arreglarlo
def extract_screen_features_fixed(screen_str):
    """Extrae características de pantalla (CORREGIDO)"""
    if pd.isna(screen_str):
        return (np.nan, 0, 0, 0)
    
    screen_str = str(screen_str)
    
    # Detectar tipos
    is_touchscreen = 1 if 'Touchscreen' in screen_str else 0
    is_ips = 1 if 'IPS' in screen_str else 0
    is_4k = 1 if '4K' in screen_str or '3840x2160' in screen_str else 0
    
    # Extraer resolución (patrón: 1920x1080)
    import re
    resolution = re.search(r'(\d{3,4})\s*x\s*(\d{3,4})', screen_str)
    
    if resolution:
        width = int(resolution.group(1))
        height = int(resolution.group(2))
        total_pixels = width * height
    else:
        total_pixels = np.nan
    
    return (total_pixels, is_touchscreen, is_ips, is_4k)

# Reaplicar
screen_features = train_processed['ScreenResolution'].apply(extract_screen_features_fixed)
train_processed['Screen_Pixels'] = [x[0] for x in screen_features]
train_processed['Is_Touchscreen'] = [x[1] for x in screen_features]
train_processed['Is_IPS'] = [x[2] for x in screen_features]
train_processed['Is_4K'] = [x[3] for x in screen_features]

screen_features_test = test_processed['ScreenResolution'].apply(extract_screen_features_fixed)
test_processed['Screen_Pixels'] = [x[0] for x in screen_features_test]
test_processed['Is_Touchscreen'] = [x[1] for x in screen_features_test]
test_processed['Is_IPS'] = [x[2] for x in screen_features_test]
test_processed['Is_4K'] = [x[3] for x in screen_features_test]

print(" Screen_Pixels corregido")
print(f"   Ejemplos: {train_processed['Screen_Pixels'].head(5).tolist()}")
print(f"   Nulos: {train_processed['Screen_Pixels'].isna().sum()}")

CORRECCIÓN: SCREEN_PIXELS
✅ Screen_Pixels corregido
   Ejemplos: [2073600, 2073600, 2073600, 1296000, 2073600]
   Nulos: 0


In [ ]:
# ============================================================================
# MEJORA: FEATURE ENGINEERING AVANZADO
# ============================================================================
print("="*70)
print("FEATURE ENGINEERING AVANZADO")
print("="*70)

import numpy as np

# Aplicar a train y test
for df in [train_processed, test_processed]:
    
    # 1. PPI (Pixels Per Inch)
    df['PPI'] = df['Screen_Pixels'] / (df['Inches'] ** 2)
    df['PPI'] = df['PPI'].fillna(df['PPI'].median())
    
    # 2. RAM por peso
    df['RAM_per_Weight'] = df['Ram_GB'] / df['Weight_kg']
    df['RAM_per_Weight'] = df['RAM_per_Weight'].fillna(df['RAM_per_Weight'].median())
    
    # 3. Storage por peso
    df['Storage_per_Weight'] = df['Memory_GB'] / df['Weight_kg']
    df['Storage_per_Weight'] = df['Storage_per_Weight'].fillna(df['Storage_per_Weight'].median())
    
    # 4. Total de recursos
    df['Total_Resources'] = df['Ram_GB'] + (df['Memory_GB'] / 100)
    
    # 5. Es gaming
    df['Is_Gaming'] = ((df['GPU_Brand'] == 'Nvidia') & (df['Ram_GB'] >= 8)).astype(int)
    
    # 6. Es premium
    df['Is_Premium'] = (
        (df['Company'] == 'Apple') | 
        ((df['Ram_GB'] >= 16) & (df['Has_SSD'] == 1))
    ).astype(int)
    
    # 7. Interacción RAM x Storage
    df['RAM_x_Storage'] = df['Ram_GB'] * np.log1p(df['Memory_GB'])
    df['RAM_x_Storage'] = df['RAM_x_Storage'].fillna(df['RAM_x_Storage'].median())

print(" Nuevas features creadas")

FEATURE ENGINEERING AVANZADO
✅ Nuevas features creadas


In [ ]:
print("="*70)
print("AÑADIENDO 'PRODUCT' A LAS FEATURES CATEGÓRICAS")
print("="*70)

# Verificar Product en ambos datasets
print("\n ANÁLISIS DE 'PRODUCT':")
print(f"   Productos únicos en TRAIN: {train_processed['Product'].nunique()}")
print(f"   Productos únicos en TEST: {test_processed['Product'].nunique()}")

# Ver si hay productos nuevos en test
products_train = set(train_processed['Product'].unique())
products_test = set(test_processed['Product'].unique())

productos_nuevos = products_test - products_train
productos_comunes = products_test & products_train

print(f"\n   Productos comunes (train ∩ test): {len(productos_comunes)}")
print(f"   Productos SOLO en train: {len(products_train - products_test)}")
print(f"   Productos NUEVOS en test: {len(productos_nuevos)}")

if len(productos_nuevos) > 0:
    print(f"\n    Hay {len(productos_nuevos)} productos en test no vistos en train")
    print(f"      Estos se encodearán como -1 (desconocido)")
else:
    print(f"\n    Todos los productos de test están en train")

AÑADIENDO 'PRODUCT' A LAS FEATURES CATEGÓRICAS

📊 ANÁLISIS DE 'PRODUCT':
   Productos únicos en TRAIN: 480
   Productos únicos en TEST: 265

   Productos comunes (train ∩ test): 127
   Productos SOLO en train: 353
   Productos NUEVOS en test: 138

   ⚠️ Hay 138 productos en test no vistos en train
      Estos se encodearán como -1 (desconocido)


In [ ]:
print("\n" + "="*70)
print("PASO 3: PREPARACIÓN DE DATOS (CON PRODUCT)")
print("="*70)



# NUEVO (con features mejoradas)
numeric_features = [
    'Inches',
    'Ram_GB',
    'Weight_kg',
    'Memory_GB',
    'Has_SSD',
    'Has_HDD',
    'Screen_Pixels',
    'Is_Touchscreen',
    'Is_IPS',
    'Is_4K',
    # NUEVAS FEATURES ⬇
    'PPI',
    'RAM_per_Weight',
    'Storage_per_Weight',
    'Total_Resources',
    'Is_Gaming',
    'Is_Premium',
    'RAM_x_Storage'
]

# Features categóricas (AHORA CON PRODUCT)
categorical_features = [
    'Company',
    'Product',      # ← AÑADIDO
    'TypeName',
    'OpSys',
    'CPU_Brand',
    'GPU_Brand'
]

print(f"\n FEATURES SELECCIONADAS:")
print(f"   Numéricas: {len(numeric_features)}")
print(f"   Categóricas: {len(categorical_features)}")
print(f"   Total: {len(numeric_features) + len(categorical_features)}")

# ============================================================================
# LABEL ENCODING DE CATEGÓRICAS
# ============================================================================
print("\n🔧 Encoding de features categóricas...")

from sklearn.preprocessing import LabelEncoder

# Diccionario para guardar los encoders
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    
    # Fit en train
    train_processed[col + '_encoded'] = le.fit_transform(train_processed[col].astype(str))
    
    # Transform en test (manejo de valores nuevos)
    def safe_transform(x, encoder):
        try:
            return encoder.transform([str(x)])[0]
        except:
            return -1  # Valor para categorías no vistas
    
    test_processed[col + '_encoded'] = test_processed[col].apply(
        lambda x: safe_transform(x, le)
    )
    
    label_encoders[col] = le
    
    n_categorias = train_processed[col].nunique()
    n_nuevas_test = (test_processed[col + '_encoded'] == -1).sum()
    
    print(f"    {col:15s}: {n_categorias:3d} categorías", end="")
    if n_nuevas_test > 0:
        print(f" |  {n_nuevas_test} nuevas en test")
    else:
        print("")

# ============================================================================
# CREAR DATASETS FINALES
# ============================================================================
print("\n Creando datasets finales...")

# Nombres de features encoded
categorical_features_encoded = [col + '_encoded' for col in categorical_features]

# Todas las features para el modelo
all_features = numeric_features + categorical_features_encoded

# X train
X_train = train_processed[all_features].copy()
y_train = train_processed['Price_in_euros'].copy()

# X test
X_test = test_processed[all_features].copy()
 DATASETS CREADOS:")
print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   X_test: {X_test.shape}")

# Verificar nulos
print(f"\n VERIFICACIÓN DE NULOS:")
nulos_train = X_train.isnull().sum().sum()
nulos_test = X_test.isnull().sum().sum()

print(f"   X_train nulos: {nulos_train}")
print(f"   X_test nulos: {nulos_test}")

# Si hay nulos, imputar con mediana
if nulos_train > 0 or nulos_test > 0:
    print("\n Imputando nulos con mediana...")
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    X_train = pd.DataFrame(
        imputer.fit_transform(X_train),
        columns=X_train.columns,
        index=X_train.index
    )
    X_test = pd.DataFrame(
        imputer.transform(X_test),
        columns=X_test.columns,
        index=X_test.index
    )
    print("    Nulos imputados")

print("\n FEATURES FINALES:")
for i, feat in enumerate(all_features, 1):
    print(f"   {i:2d}. {feat}")


PASO 3: PREPARACIÓN DE DATOS (CON PRODUCT)

📊 FEATURES SELECCIONADAS:
   Numéricas: 17
   Categóricas: 6
   Total: 23

🔧 Encoding de features categóricas...
   ✅ Company        :  19 categorías
   ✅ Product        : 480 categorías | ⚠️ 146 nuevas en test
   ✅ TypeName       :   6 categorías
   ✅ OpSys          :   9 categorías
   ✅ CPU_Brand      :   5 categorías | ⚠️ 1 nuevas en test
   ✅ GPU_Brand      :   3 categorías | ⚠️ 1 nuevas en test

📊 Creando datasets finales...

✅ DATASETS CREADOS:
   X_train: (912, 23)
   y_train: (912,)
   X_test: (391, 23)

🔍 VERIFICACIÓN DE NULOS:
   X_train nulos: 12
   X_test nulos: 8

⚠️ Imputando nulos con mediana...
   ✅ Nulos imputados

📋 FEATURES FINALES:
    1. Inches
    2. Ram_GB
    3. Weight_kg
    4. Memory_GB
    5. Has_SSD
    6. Has_HDD
    7. Screen_Pixels
    8. Is_Touchscreen
    9. Is_IPS
   10. Is_4K
   11. PPI
   12. RAM_per_Weight
   13. Storage_per_Weight
   14. Total_Resources
   15. Is_Gaming
   16. Is_Premium
   17. RAM_x_Sto

In [4]:
# ============================================================================
# PASO 5.1: BASELINE DE MODELOS
# ============================================================================
print("="*70)
print("PASO 5.1: BASELINE - COMPARACIÓN DE MODELOS")
print("="*70)

print("""
 ¿QUÉ ES UN BASELINE?

Un baseline es un modelo SIMPLE que sirve de referencia:
  • Si tu modelo complejo NO supera al baseline → Problema
  • Si tu modelo complejo SÍ supera al baseline → Vas bien
  
Probamos:
  1. Linear Regression (más simple posible)
  2. Random Forest (robusto, medio complejo)
  3. LightGBM (complejo, mejor rendimiento esperado)
""")

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Configurar Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ============================================================================
# BASELINE 1: LINEAR REGRESSION
# ============================================================================
print("\\n1️⃣ BASELINE 1: Linear Regression")
print("-" * 70)

lr = LinearRegression()

print("    Entrenando con Cross-Validation (5-fold)...")
lr_scores = cross_val_score(
    lr, 
    X_train, 
    y_train, 
    cv=kf, 
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

lr_rmse = -lr_scores.mean()
lr_std = lr_scores.std()

print(f"    Completado")
print(f"\\n    RESULTADOS:")
print(f"      RMSE: {lr_rmse:.2f} ± {lr_std:.2f}")

# ============================================================================
# BASELINE 2: RANDOM FOREST
# ============================================================================
print("\\n2️⃣ BASELINE 2: Random Forest")
print("-" * 70)

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("    Entrenando con Cross-Validation (5-fold)...")
print("   (Esto puede tardar 30-60 segundos...)")

rf_scores = cross_val_score(
    rf, 
    X_train, 
    y_train, 
    cv=kf,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

rf_rmse = -rf_scores.mean()
rf_std = rf_scores.std()

print(f"    Completado")
print(f"\\n    RESULTADOS:")
print(f"      RMSE: {rf_rmse:.2f} ± {rf_std:.2f}")

# ============================================================================
# COMPARACIÓN DE BASELINES
# ============================================================================
print("\\n" + "="*70)
print(" COMPARACIÓN DE BASELINES")
print("="*70)





PASO 5.1: BASELINE - COMPARACIÓN DE MODELOS

 ¿QUÉ ES UN BASELINE?

Un baseline es un modelo SIMPLE que sirve de referencia:
  • Si tu modelo complejo NO supera al baseline → Problema
  • Si tu modelo complejo SÍ supera al baseline → Vas bien

Probamos:
  1. Linear Regression (más simple posible)
  2. Random Forest (robusto, medio complejo)
  3. LightGBM (complejo, mejor rendimiento esperado)

\n1️⃣ BASELINE 1: Linear Regression
----------------------------------------------------------------------
    Entrenando con Cross-Validation (5-fold)...


NameError: name 'X_train' is not defined

In [ ]:
print("\n" + "="*70)
print("PASO 4: ENTRENAMIENTO DE LIGHTGBM")
print("="*70)

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# ============================================================================
# CONFIGURACIÓN DEL MODELO
# ============================================================================
print("\n Configurando LightGBM...")

# NUEVO (tunning mejorado)
model = lgb.LGBMRegressor(
    n_estimators=2000,      # ⬆ Más árboles
    learning_rate=0.03,     # ⬇ Learning rate más bajo
    max_depth=8,            # ⬆ Árboles más profundos
    num_leaves=40,          # ⬆ Más hojas
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

print(" Modelo configurado")

# ============================================================================
# CROSS-VALIDATION
# ============================================================================
print("\n Realizando Cross-Validation (5-fold)...")
print("   (Esto puede tardar 1-2 minutos...)")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    model, 
    X_train, 
    y_train, 
    cv=kf, 
    scoring='neg_root_mean_squared_error',  # RMSE
    n_jobs=-1
)

# Convertir a positivo
cv_rmse = -cv_scores

print(f"\n RESULTADOS DE CROSS-VALIDATION:")
print(f"   RMSE por fold:")
for i, score in enumerate(cv_rmse, 1):
    print(f"      Fold {i}: {score:.2f}")

print(f"\n   {'='*50}")
print(f"    RMSE MEDIO: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}")
print(f"   {'='*50}")

if cv_rmse.mean() < 400:
    print(f"\n   ¡OBJETIVO ALCANZADO! (RMSE < 400)")
elif cv_rmse.mean() < 450:
    print(f"\n    Cerca del objetivo (tunear hiperparámetros)")
else:
    print(f"\n    Lejos del objetivo (revisar features)")

# ============================================================================
# ENTRENAR CON TODOS LOS DATOS
# ============================================================================
print("\n Entrenando modelo final con todos los datos...")

model.fit(X_train, y_train)

print(" Modelo entrenado")

# RMSE en train (para referencia)
y_train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

print(f"\n RMSE EN TRAIN: {train_rmse:.2f}")
print(f"   (Referencia - El RMSE real es el de CV: {cv_rmse.mean():.2f})")

# ============================================================================
# IMPORTANCIA DE FEATURES
# ============================================================================
print("\n TOP 15 FEATURES MÁS IMPORTANTES:")

feature_importance = pd.DataFrame({
    'feature': all_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(15).to_string(index=False))


PASO 4: ENTRENAMIENTO DE LIGHTGBM

🤖 Configurando LightGBM...
✅ Modelo configurado

📊 Realizando Cross-Validation (5-fold)...
   (Esto puede tardar 1-2 minutos...)

📊 RESULTADOS DE CROSS-VALIDATION:
   RMSE por fold:
      Fold 1: 303.88
      Fold 2: 271.23
      Fold 3: 271.75
      Fold 4: 257.40
      Fold 5: 324.62

   🎯 RMSE MEDIO: 285.78 ± 24.70

   ✅ ¡OBJETIVO ALCANZADO! (RMSE < 400)

🚀 Entrenando modelo final con todos los datos...
✅ Modelo entrenado

📊 RMSE EN TRAIN: 75.54
   (Referencia - El RMSE real es el de CV: 285.78)

📊 TOP 15 FEATURES MÁS IMPORTANTES:
           feature  importance
   Product_encoded        5732
    RAM_per_Weight        4686
         Weight_kg        4560
Storage_per_Weight        4274
   Company_encoded        1599
   Total_Resources        1445
               PPI        1441
 CPU_Brand_encoded        1022
            Inches         959
     RAM_x_Storage         956
     Screen_Pixels         938
  TypeName_encoded         733
     OpSys_encoded   

In [ ]:
print("\n" + "="*70)
print("PASO 5: PREDICCIONES Y SUBMISSION")
print("="*70)

# Predecir en test
print("\n Generando predicciones para test...")
predictions = model.predict(X_test)

print(f" Predicciones generadas: {len(predictions)}")
print(f"\n ESTADÍSTICAS DE PREDICCIONES:")
print(f"   Pecio mínimo predicho: {predictions.min():.2f} €")
print(f"   Precio máximo predicho: {predictions.max():.2f} €")
print(f"   Precio medio predicho: {predictions.mean():.2f} €")
print(f"   Precio mediano predicho: {np.median(predictions):.2f} €")

# Crear submission
submission = pd.DataFrame({
    'laptop_ID': test_processed['laptop_ID'],
    'Price_in_euros': predictions
})

# Guardar
submission.to_csv('submission_improved.csv', index=False)

print(f"\n ARCHIVO GUARDADO: submission.csv")
print(f"\n Primeras 10 predicciones:")
print(submission.head(10).to_string(index=False))

print(f"""
═══════════════════════════════════════════════════════════════════════
 PROCESO COMPLETADO
═══════════════════════════════════════════════════════════════════════

 RESUMEN FINAL:
   • Features usadas: {len(all_features)} (10 numéricas + 6 categóricas)
   • Incluye: Product (480 categorías)
   • RMSE esperado (CV): {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}
   • Objetivo Kaggle: < 400
   • Estado: {' ALCANZADO' if cv_rmse.mean() < 400 else '⚠️ REVISAR'}

 ARCHIVOS GENERADOS:
   • submission.csv ({len(submission)} predicciones)

 PRÓXIMOS PASOS:
   1. Sube submission.csv a Kaggle
   2. Compara RMSE del leaderboard con CV ({cv_rmse.mean():.2f})
   3. Si coinciden → Todo correcto 
   4. Si RMSE Kaggle > CV → Posible overfitting
   5. Si RMSE Kaggle < CV → ¡Mejor de lo esperado! 🎉

 SI RMSE > 400:
   • Tunear hiperparámetros (aumentar n_estimators, bajar learning_rate)
   • Crear features de interacción (PPI, Gaming, etc.)
   • Probar ensemble de modelos
""")


PASO 5: PREDICCIONES Y SUBMISSION

🔮 Generando predicciones para test...
✅ Predicciones generadas: 391

📊 ESTADÍSTICAS DE PREDICCIONES:
   Precio mínimo predicho: 221.22 €
   Precio máximo predicho: 5535.99 €
   Precio medio predicho: 1126.42 €
   Precio mediano predicho: 965.59 €

💾 ARCHIVO GUARDADO: submission.csv

📋 Primeras 10 predicciones:
 laptop_ID  Price_in_euros
       209     1143.228502
      1281      289.335853
      1168      344.374493
      1231      820.365899
      1020     1054.912451
       379      528.876922
       553      692.498873
       172     1094.424555
       779     1067.763729
       609      322.520844

═══════════════════════════════════════════════════════════════════════
✅ PROCESO COMPLETADO
═══════════════════════════════════════════════════════════════════════

📊 RESUMEN FINAL:
   • Features usadas: 23 (10 numéricas + 6 categóricas)
   • Incluye: Product (480 categorías)
   • RMSE esperado (CV): 285.78 ± 24.70
   • Objetivo Kaggle: < 400
   • Est

In [ ]:
print("\n VERIFICACIÓN FINAL DE SUBMISSION.CSV:")


# Leer el archivo generado
submission_check = pd.read_csv('submission.csv')

print(f"\n FORMATO DEL ARCHIVO:")
print(f"   Columnas: {submission_check.columns.tolist()}")
print(f"   Filas: {len(submission_check)}")
print(f"   Columna 1 (ID): {submission_check.columns[0]}")
print(f"   Columna 2 (Precio): {submission_check.columns[1]}")

print(f"\n PRIMERAS 5 FILAS:")
print(submission_check.head())

print(f"\n ÚLTIMAS 5 FILAS:")
print(submission_check.tail())

# Verificar que no hay nulos
nulos_submission = submission_check.isnull().sum().sum()
print(f"\n Valores nulos en submission: {nulos_submission}")

if nulos_submission == 0:
    print("    No hay nulos - Archivo listo para subir")
else:
    print("    HAY NULOS - REVISAR")

# Verificar IDs
ids_unicos = submission_check['laptop_ID'].nunique()
print(f"\n🔍 IDs únicos: {ids_unicos}")

if ids_unicos == len(submission_check):
    print("    Todos los IDs son únicos")
else:
    print("    Hay IDs duplicados")




✅ VERIFICACIÓN FINAL DE SUBMISSION.CSV:

📋 FORMATO DEL ARCHIVO:
   Columnas: ['laptop_ID', 'Price_in_euros']
   Filas: 391
   Columna 1 (ID): laptop_ID
   Columna 2 (Precio): Price_in_euros

📊 PRIMERAS 5 FILAS:
   laptop_ID  Price_in_euros
0        209     1194.910244
1       1281      308.557594
2       1168      308.225638
3       1231      905.020887
4       1020     1050.592808

📊 ÚLTIMAS 5 FILAS:
     laptop_ID  Price_in_euros
386        820     2276.204500
387        948      829.150096
388        483     1288.557235
389       1017      937.300641
390        421     1192.239609

🔍 Valores nulos en submission: 0
   ✅ No hay nulos - Archivo listo para subir

🔍 IDs únicos: 391
   ✅ Todos los IDs son únicos

═══════════════════════════════════════════════════════════════════════
🎉 ¡TODO LISTO PARA SUBIR A KAGGLE!
═══════════════════════════════════════════════════════════════════════

ARCHIVO: submission.csv ✅
RMSE ESPERADO: ~275.67 (muy por debajo de 400) ✅
FORMATO CORRECTO: Sí ✅



In [5]:
# ============================================================================
# PASO 6: VERIFICAR Y GUARDAR SUBMISSION
# ============================================================================
print("="*70)
print("PASO 6: VERIFICAR Y GUARDAR SUBMISSION")
print("="*70)

# Cargar sample_submission con la ruta correcta
sample_submission = pd.read_csv('data/sample_submission.csv')  # ← RUTA CORRECTA

print("\n VERIFICANDO SUBMISSION...")

# Check 1: Shape
print(f"\n1️ Shape:")
print(f"   Tu submission: {submission.shape}")
print(f"   Esperado: {sample_submission.shape}")
if submission.shape == sample_submission.shape:
    print(f"    Correcto")
else:
    print(f"    Incorrecto")

# Check 2: Columnas
print(f"\n2️⃣ Columnas:")
print(f"   Tus columnas: {submission.columns.tolist()}")
print(f"   Esperadas: {sample_submission.columns.tolist()}")
if submission.columns.tolist() == sample_submission.columns.tolist():
    print(f"    Correcto")
else:
    print(f"    Incorrecto")

# Check 3: IDs
print(f"\n3️⃣ IDs:")
if submission['laptop_ID'].tolist() == sample_submission['laptop_ID'].tolist():
    print(f"    IDs correctos ({len(submission)} laptops)")
else:
    print(f"    IDs no coinciden")

# Check 4: Nulos
print(f"\n4️⃣ Valores nulos:")
nulos = submission.isnull().sum().sum()
print(f"   Nulos encontrados: {nulos}")
if nulos == 0:
    print(f"    Sin valores nulos")
else:
    print(f"    Hay nulos - CORREGIR")

# Check 5: Tipos
print(f"\n5️⃣ Tipos de datos:")
print(f"   Price_in_euros: {submission['Price_in_euros'].dtype}")
if submission['Price_in_euros'].dtype in ['float64', 'float32']:
    print(f"    Tipo correcto")
else:
    print(f"    Debe ser float")

# Si todo OK, guardar
all_ok = (
    submission.shape == sample_submission.shape and
    submission.columns.tolist() == sample_submission.columns.tolist() and
    submission['laptop_ID'].tolist() == sample_submission['laptop_ID'].tolist() and
    nulos == 0 and
    submission['Price_in_euros'].dtype in ['float64', 'float32']
)

if all_ok:
    print("\n" + "="*70)
    print(" ¡TODAS LAS VERIFICACIONES PASADAS!")
    print("="*70)
    
    # Guardar
    submission.to_csv('submission.csv', index=False)
    
    print(f"\ ARCHIVO GUARDADO: submission.csv")
    print(f"\n RESUMEN:")
    print(f"   • Filas: {len(submission)}")
    print(f"   • Columnas: {list(submission.columns)}")
    print(f"   • RMSE esperado (CV): ~275")
    
 


PASO 6: VERIFICAR Y GUARDAR SUBMISSION


<>:72: SyntaxWarning: invalid escape sequence '\ '
<>:72: SyntaxWarning: invalid escape sequence '\ '
C:\Users\lupep\AppData\Local\Temp\ipykernel_14124\3435053973.py:72: SyntaxWarning: invalid escape sequence '\ '
  print(f"\ ARCHIVO GUARDADO: submission.csv")
C:\Users\lupep\AppData\Local\Temp\ipykernel_14124\3435053973.py:72: SyntaxWarning: invalid escape sequence '\ '
  print(f"\ ARCHIVO GUARDADO: submission.csv")


NameError: name 'pd' is not defined

In [2]:



# RE-CARGAR con rutas explícitas
print("\n Cargando archivos desde carpeta 'data'...")

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

print(f"\n Archivos cargados")

print(f"\n VERIFICACIÓN:")
print(f"   train.csv: {train.shape} | Tiene precio: {'Price_in_euros' in train.columns}")
print(f"   test.csv: {test.shape} | Tiene precio: {'Price_in_euros' in test.columns}")
print(f"   sample_submission.csv: {sample_submission.shape}")

# Ver primeros IDs
print(f"\n PRIMEROS 3 IDs:")
print(f"   train: {train['laptop_ID'].head(3).tolist()}")
print(f"   test: {test['laptop_ID'].head(3).tolist()}")
print(f"   sample: {sample_submission['laptop_ID'].head(3).tolist()}")

# Verificar que test y sample tengan los mismos IDs
if test['laptop_ID'].tolist() == sample_submission['laptop_ID'].tolist():
    print(f"\n    test.csv y sample_submission.csv tienen los MISMOS IDs")
else:
    print(f"\n    test.csv y sample_submission.csv tienen IDs DIFERENTES")
    print(f"\n    PROBLEMA: Los archivos no coinciden")




 Cargando archivos desde carpeta 'data'...


NameError: name 'pd' is not defined

In [1]:
# ============================================================================
# ENSEMBLE COMPLETO - VERSIÓN INDEPENDIENTE
# ============================================================================


print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   X_test: {X_test.shape}")

# Instalar XGBoost si no está (ejecutar solo si da error)
# !pip install xgboost

from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
import lightgbm as lgb
import numpy as np



# Modelos base
lgb_model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=8,
    num_leaves=40,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

xgb_model = xgb.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

meta_model = Ridge(alpha=1.0)

ensemble = StackingRegressor(
    estimators=[
        ('lightgbm', lgb_model),
        ('xgboost', xgb_model),
        ('random_forest', rf_model)
    ],
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print(" Modelos configurados")

# Cross-Validation (opcional, comentar si tarda mucho)
print("\n Evaluando con CV (3-fold)...")
print("    Esperando 3-5 minutos...")

kf = KFold(n_splits=3, shuffle=True, random_state=42)

try:
    cv_scores = cross_val_score(
        ensemble, X_train, y_train,
        cv=kf,
        scoring='neg_root_mean_squared_error'
    )
    print(f"\n CV completado")
    print(f"   RMSE (CV): {-cv_scores.mean():.2f} ± {cv_scores.std():.2f}")
except:
    print("    CV omitido (para ir más rápido)")

# Entrenar
print("\n Entrenando ensemble final...")
ensemble.fit(X_train, y_train)
print(" Entrenado")

# Predecir
print("\n Prediciendo...")
predictions_ensemble = ensemble.predict(X_test)

# Submission
submission_ensemble = pd.DataFrame({
    'laptop_ID': test_processed['laptop_ID'],
    'Price_in_euros': predictions_ensemble
})

submission_ensemble.to_csv('submission_ensemble.csv', index=False)

print(f"\n{'='*70}")
print(f" ¡COMPLETADO!")
print(f"{'='*70}")
print(f"\n Archivo: submission_ensemble.csv")
print(f" Predicciones: {len(submission_ensemble)}")
print(f" LISTO PARA SUBIR A KAGGLE")
print(f"{'='*70}")

NameError: name 'X_train' is not defined

### 2.3 Definir X e y

In [22]:
X = df.drop(['Price_in_euros'], axis=1)
y = df['Price_in_euros'].copy()
X.shape

KeyError: "['Price_in_euros'] not found in axis"

In [ ]:
y.shape

### 2.4 Dividir X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
X_train

In [ ]:
y_train

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

-----------------------------------------------------------------------------------------------------------------

## 4. Modelado

### 4.1 Baseline de modelos


### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

### 4.3 Optimización (up to you 🫰🏻)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [ ]:
X_pred = pd.read_csv("./data/test.csv")
X_pred.head()

In [ ]:
X_pred.tail()

In [ ]:
X_pred.info()

 ## 2. Replicar el procesado para ``test.csv``

In [ ]:
X_pred

In [ ]:
predictions_submit = model.predict(X_pred)
predictions_submit

**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [ ]:
# ¿Qué opináis?
# ¿Sí, no?

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [ ]:
sample = pd.read_csv("data/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
sample.shape

## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [ ]:
#¿Cómo creamos la submission?
submission = pd.DataFrame()

In [ ]:
submission.head()

In [ ]:
submission.shape

## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [ ]:
chequeador(submission)

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# ============================================================================
# PASO 1: CARGAR DATOS
# ============================================================================
print("="*70)
print("PASO 1: CARGA DE DATOS")
print("="*70)

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

print(f"\n Datos cargados:")
print(f"   train: {train.shape}")
print(f"   test: {test.shape}")
print(f"   sample: {sample_submission.shape}")

train_processed = train.copy()
test_processed = test.copy()

# ============================================================================
# PASO 2: FEATURE ENGINEERING
# ============================================================================
print("\n" + "="*70)
print("PASO 2: FEATURE ENGINEERING")
print("="*70)

import re

# 1. RAM
def extract_ram(ram_str):
    if pd.isna(ram_str):
        return np.nan
    numbers = re.findall(r'\d+', str(ram_str))
    return int(numbers[0]) if numbers else np.nan

train_processed['Ram_GB'] = train_processed['Ram'].apply(extract_ram)
test_processed['Ram_GB'] = test_processed['Ram'].apply(extract_ram)

# 2. WEIGHT
def extract_weight(weight_str):
    if pd.isna(weight_str):
        return np.nan
    numbers = re.findall(r'\d+\.?\d*', str(weight_str))
    return float(numbers[0]) if numbers else np.nan

train_processed['Weight_kg'] = train_processed['Weight'].apply(extract_weight)
test_processed['Weight_kg'] = test_processed['Weight'].apply(extract_weight)

# 3. MEMORY
def extract_memory_features(memory_str):
    if pd.isna(memory_str):
        return (np.nan, 0, 0)
    memory_str = str(memory_str).upper()
    has_ssd = 1 if 'SSD' in memory_str else 0
    has_hdd = 1 if 'HDD' in memory_str else 0
    total_gb = 0
    gb_matches = re.findall(r'(\d+)\s*GB', memory_str)
    tb_matches = re.findall(r'(\d+)\s*TB', memory_str)
    for gb in gb_matches:
        total_gb += int(gb)
    for tb in tb_matches:
        total_gb += int(tb) * 1024
    return (total_gb if total_gb > 0 else np.nan, has_ssd, has_hdd)

memory_train = train_processed['Memory'].apply(extract_memory_features)
train_processed['Memory_GB'] = [x[0] for x in memory_train]
train_processed['Has_SSD'] = [x[1] for x in memory_train]
train_processed['Has_HDD'] = [x[2] for x in memory_train]

memory_test = test_processed['Memory'].apply(extract_memory_features)
test_processed['Memory_GB'] = [x[0] for x in memory_test]
test_processed['Has_SSD'] = [x[1] for x in memory_test]
test_processed['Has_HDD'] = [x[2] for x in memory_test]

# 4. SCREEN
def extract_screen_features(screen_str):
    if pd.isna(screen_str):
        return (np.nan, 0, 0, 0)
    screen_str = str(screen_str)
    is_touchscreen = 1 if 'Touchscreen' in screen_str else 0
    is_ips = 1 if 'IPS' in screen_str else 0
    is_4k = 1 if '4K' in screen_str or '3840x2160' in screen_str else 0
    resolution = re.search(r'(\d{3,4})\s*x\s*(\d{3,4})', screen_str)
    if resolution:
        width = int(resolution.group(1))
        height = int(resolution.group(2))
        total_pixels = width * height
    else:
        total_pixels = np.nan
    return (total_pixels, is_touchscreen, is_ips, is_4k)

screen_train = train_processed['ScreenResolution'].apply(extract_screen_features)
train_processed['Screen_Pixels'] = [x[0] for x in screen_train]
train_processed['Is_Touchscreen'] = [x[1] for x in screen_train]
train_processed['Is_IPS'] = [x[2] for x in screen_train]
train_processed['Is_4K'] = [x[3] for x in screen_train]

screen_test = test_processed['ScreenResolution'].apply(extract_screen_features)
test_processed['Screen_Pixels'] = [x[0] for x in screen_test]
test_processed['Is_Touchscreen'] = [x[1] for x in screen_test]
test_processed['Is_IPS'] = [x[2] for x in screen_test]
test_processed['Is_4K'] = [x[3] for x in screen_test]

# 5. CPU
def extract_cpu_brand(cpu_str):
    if pd.isna(cpu_str):
        return 'Unknown'
    cpu_str = str(cpu_str).upper()
    if 'INTEL CORE I7' in cpu_str:
        return 'Intel_i7'
    elif 'INTEL CORE I5' in cpu_str:
        return 'Intel_i5'
    elif 'INTEL CORE I3' in cpu_str:
        return 'Intel_i3'
    elif 'AMD' in cpu_str:
        return 'AMD'
    elif 'INTEL' in cpu_str:
        return 'Intel_Other'
    else:
        return 'Other'

train_processed['CPU_Brand'] = train_processed['Cpu'].apply(extract_cpu_brand)
test_processed['CPU_Brand'] = test_processed['Cpu'].apply(extract_cpu_brand)

# 6. GPU
def extract_gpu_brand(gpu_str):
    if pd.isna(gpu_str):
        return 'Unknown'
    gpu_str = str(gpu_str).upper()
    if 'NVIDIA' in gpu_str or 'GEFORCE' in gpu_str:
        return 'Nvidia'
    elif 'AMD' in gpu_str or 'RADEON' in gpu_str:
        return 'AMD'
    elif 'INTEL' in gpu_str:
        return 'Intel'
    else:
        return 'Other'

train_processed['GPU_Brand'] = train_processed['Gpu'].apply(extract_gpu_brand)
test_processed['GPU_Brand'] = test_processed['Gpu'].apply(extract_gpu_brand)

print(" Feature Engineering completado")

# ============================================================================
# PASO 3: PREPARAR DATOS
# ============================================================================
print("\n" + "="*70)
print("PASO 3: PREPARACIÓN DE DATOS")
print("="*70)

# Features numéricas
numeric_features = [
    'Inches',
    'Ram_GB',
    'Weight_kg',
    'Memory_GB',
    'Has_SSD',
    'Has_HDD',
    'Screen_Pixels',
    'Is_Touchscreen',
    'Is_IPS',
    'Is_4K'
]

# Features categóricas
categorical_features = [
    'Company',
    'Product',
    'TypeName',
    'OpSys',
    'CPU_Brand',
    'GPU_Brand'
]

# Label Encoding
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    train_processed[col + '_encoded'] = le.fit_transform(train_processed[col].astype(str))
    
    def safe_transform(x, encoder):
        try:
            return encoder.transform([str(x)])[0]
        except:
            return -1
    
    test_processed[col + '_encoded'] = test_processed[col].apply(lambda x: safe_transform(x, le))
    label_encoders[col] = le

# Features finales
categorical_features_encoded = [col + '_encoded' for col in categorical_features]
all_features = numeric_features + categorical_features_encoded

# Crear datasets
X_train = train_processed[all_features].copy()
y_train = train_processed['Price_in_euros'].copy()
X_test = test_processed[all_features].copy()

# Imputar nulos si hay
if X_train.isnull().sum().sum() > 0 or X_test.isnull().sum().sum() > 0:
    imputer = SimpleImputer(strategy='median')
    X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

print(f"\n Datasets creados:")
print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   X_test: {X_test.shape}")

# ============================================================================
# PASO 4: MODELO LIGHTGBM
# ============================================================================
print("\n" + "="*70)
print("PASO 4: ENTRENAMIENTO DE LIGHTGBM")
print("="*70)

model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, 
                             scoring='neg_root_mean_squared_error', n_jobs=-1)
cv_rmse = -cv_scores

print(f"\n CROSS-VALIDATION:")
for i, score in enumerate(cv_rmse, 1):
    print(f"   Fold {i}: {score:.2f}")
print(f"\n    RMSE MEDIO: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}")

# Entrenar con todos los datos
model.fit(X_train, y_train)
print(f"\n Modelo entrenado")

# ============================================================================
# PASO 5: PREDICCIONES
# ============================================================================
print("\n" + "="*70)
print("PASO 5: PREDICCIONES")
print("="*70)

predictions = model.predict(X_test)

print(f" Predicciones generadas: {len(predictions)}")
print(f"   Min: {predictions.min():.2f} €")
print(f"   Max: {predictions.max():.2f} €")
print(f"   Media: {predictions.mean():.2f} €")

# Crear submission
submission = pd.DataFrame({
    'laptop_ID': test_processed['laptop_ID'],
    'Price_in_euros': predictions
})

print(f"\n Submission creado: {submission.shape}")

# ============================================================================
# PASO 6: VERIFICAR Y GUARDAR
# ============================================================================
print("\n" + "="*70)
print("PASO 6: VERIFICAR Y GUARDAR")
print("="*70)

# Verificaciones
checks_passed = True

if submission.shape == sample_submission.shape:
    print(f"    Shape correcto: {submission.shape}")
else:
    print(f"    Shape incorrecto: {submission.shape} vs {sample_submission.shape}")
    checks_passed = False

if submission.columns.tolist() == sample_submission.columns.tolist():
    print(f"    Columnas correctas")
else:
    print(f"    Columnas incorrectas")
    checks_passed = False

if submission['laptop_ID'].tolist() == sample_submission['laptop_ID'].tolist():
    print(f"    IDs correctos")
else:
    print(f"    IDs no coinciden")
    checks_passed = False

if submission.isnull().sum().sum() == 0:
    print(f"    Sin valores nulos")
else:
    print(f"    Hay valores nulos")
    checks_passed = False

if checks_passed:
    submission.to_csv('submission.csv', index=False)
    print(f"\n{'='*70}")
    print(f" ¡SUBMISSION GUARDADO CORRECTAMENTE!")
    print(f"{'='*70}")
    print(f"\n Archivo: submission.csv")
    print(f" Filas: {len(submission)}")
    print(f" RMSE esperado: ~{cv_rmse.mean():.2f}")
    print(f"\n¡Listo para subir a Kaggle! 🚀")
else:
    print(f"\n Hay errores - Revisar")

